###Loading Dataset and Basic preprocessing

In [76]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid/regression/Data"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid/regression/Data


In [2]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd
import os

In [154]:
#RUNNING IN JUPYTER
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\HRV_features")
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Padasip_Clean_Data")

In [155]:
def add_baseline_fatigue_and_delta(df):
    # Create a dictionary to hold the baseline fatigue rating for each subject
    baseline_fatigue_dict = {}

    # Loop through each subject and get their baseline fatigue rating (Block 0)
    for subj_id in df['SubjID'].unique():
        baseline_fatigue_values = df[(df['SubjID'] == subj_id) & (df['Block'] == 0)]['Fatigue_Rating'].values
        if len(baseline_fatigue_values) > 0:
            baseline_fatigue_dict[subj_id] = baseline_fatigue_values[0]
        else:
            baseline_fatigue_dict[subj_id] = None  # or set a default value if needed

    # Map the baseline fatigue rating to a new column in the dataframe
    df['Baseline_Fatigue_Rating'] = df['SubjID'].map(baseline_fatigue_dict)
    
    # Calculate Delta_fatigue and Change_from_Baseline for each row
    #df['Delta_fatigue'] = df.groupby('SubjID')['Fatigue_Rating'].diff().fillna(0)
    #df['Change_from_Baseline'] = df['Fatigue_Rating'] - df['Baseline_Fatigue_Rating']

    return df

In [156]:
# f_path = "Only_HRVDelta_BlockLevel.xlsx"
f_path = "HRV_features_BlockLevel_withDelta.xlsx"
data_block = pd.read_excel(f_path, header=0)

# Renaming the 'Fatigue_Rating' column to 'Fatigue_Score'
data_block.rename(columns={'Fatigue_Rating': 'fatigue_rating'}, inplace=True)

data_block.rename(columns={'Change_from_Baseline_Fatigue_Rating': 'Change_from_Baseline'}, inplace=True)
data_block.head()
data_block.columns.tolist()

['Unnamed: 0',
 'Block',
 'SubjID',
 'fatigue_rating',
 'Age',
 'HRV_MeanNN',
 'HRV_SDNN',
 'HRV_SDANN1',
 'HRV_SDNNI1',
 'HRV_SDANN2',
 'HRV_SDNNI2',
 'HRV_SDANN5',
 'HRV_SDNNI5',
 'HRV_RMSSD',
 'HRV_SDSD',
 'HRV_CVNN',
 'HRV_CVSD',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_MCVNN',
 'HRV_IQRNN',
 'HRV_SDRMSSD',
 'HRV_Prc20NN',
 'HRV_Prc80NN',
 'HRV_pNN50',
 'HRV_pNN20',
 'HRV_MinNN',
 'HRV_MaxNN',
 'HRV_HTI',
 'HRV_TINN',
 'HRV_ULF',
 'HRV_VLF',
 'HRV_LF',
 'HRV_HF',
 'HRV_VHF',
 'HRV_TP',
 'HRV_LFHF',
 'HRV_LFn',
 'HRV_HFn',
 'HRV_LnHF',
 'HRV_SD1',
 'HRV_SD2',
 'HRV_SD1SD2',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVI',
 'HRV_CSI_Modified',
 'HRV_PIP',
 'HRV_IALS',
 'HRV_PSS',
 'HRV_PAS',
 'HRV_GI',
 'HRV_SI',
 'HRV_AI',
 'HRV_PI',
 'HRV_C1d',
 'HRV_C1a',
 'HRV_SD1d',
 'HRV_SD1a',
 'HRV_C2d',
 'HRV_C2a',
 'HRV_SD2d',
 'HRV_SD2a',
 'HRV_Cd',
 'HRV_Ca',
 'HRV_SDNNd',
 'HRV_SDNNa',
 'HRV_DFA_alpha1',
 'HRV_MFDFA_alpha1_Width',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_MFDFA_alpha1_Max',

In [157]:
# Apply the function to the dataframe
data_block = add_baseline_fatigue(data_block)
data_block.columns.tolist()

['Unnamed: 0',
 'Block',
 'SubjID',
 'fatigue_rating',
 'Age',
 'HRV_MeanNN',
 'HRV_SDNN',
 'HRV_SDANN1',
 'HRV_SDNNI1',
 'HRV_SDANN2',
 'HRV_SDNNI2',
 'HRV_SDANN5',
 'HRV_SDNNI5',
 'HRV_RMSSD',
 'HRV_SDSD',
 'HRV_CVNN',
 'HRV_CVSD',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_MCVNN',
 'HRV_IQRNN',
 'HRV_SDRMSSD',
 'HRV_Prc20NN',
 'HRV_Prc80NN',
 'HRV_pNN50',
 'HRV_pNN20',
 'HRV_MinNN',
 'HRV_MaxNN',
 'HRV_HTI',
 'HRV_TINN',
 'HRV_ULF',
 'HRV_VLF',
 'HRV_LF',
 'HRV_HF',
 'HRV_VHF',
 'HRV_TP',
 'HRV_LFHF',
 'HRV_LFn',
 'HRV_HFn',
 'HRV_LnHF',
 'HRV_SD1',
 'HRV_SD2',
 'HRV_SD1SD2',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVI',
 'HRV_CSI_Modified',
 'HRV_PIP',
 'HRV_IALS',
 'HRV_PSS',
 'HRV_PAS',
 'HRV_GI',
 'HRV_SI',
 'HRV_AI',
 'HRV_PI',
 'HRV_C1d',
 'HRV_C1a',
 'HRV_SD1d',
 'HRV_SD1a',
 'HRV_C2d',
 'HRV_C2a',
 'HRV_SD2d',
 'HRV_SD2a',
 'HRV_Cd',
 'HRV_Ca',
 'HRV_SDNNd',
 'HRV_SDNNa',
 'HRV_DFA_alpha1',
 'HRV_MFDFA_alpha1_Width',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_MFDFA_alpha1_Max',

In [158]:
# f_path = "Only_HRVDelta_30seconds.xlsx"
f_path = "HRV_features_30seconds_withDelta.xlsx"
data30 = pd.read_excel(f_path, header=0)
data30.columns.tolist()

['Block',
 'fatigue_rating',
 'window_num_30',
 'SubjID',
 'Age',
 'HRV_MeanNN',
 'HRV_SDNN',
 'HRV_SDANN1',
 'HRV_SDNNI1',
 'HRV_SDANN2',
 'HRV_SDNNI2',
 'HRV_SDANN5',
 'HRV_SDNNI5',
 'HRV_RMSSD',
 'HRV_SDSD',
 'HRV_CVNN',
 'HRV_CVSD',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_MCVNN',
 'HRV_IQRNN',
 'HRV_SDRMSSD',
 'HRV_Prc20NN',
 'HRV_Prc80NN',
 'HRV_pNN50',
 'HRV_pNN20',
 'HRV_MinNN',
 'HRV_MaxNN',
 'HRV_HTI',
 'HRV_TINN',
 'HRV_ULF',
 'HRV_VLF',
 'HRV_LF',
 'HRV_HF',
 'HRV_VHF',
 'HRV_TP',
 'HRV_LFHF',
 'HRV_LFn',
 'HRV_HFn',
 'HRV_LnHF',
 'HRV_SD1',
 'HRV_SD2',
 'HRV_SD1SD2',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVI',
 'HRV_CSI_Modified',
 'HRV_PIP',
 'HRV_IALS',
 'HRV_PSS',
 'HRV_PAS',
 'HRV_GI',
 'HRV_SI',
 'HRV_AI',
 'HRV_PI',
 'HRV_C1d',
 'HRV_C1a',
 'HRV_SD1d',
 'HRV_SD1a',
 'HRV_C2d',
 'HRV_C2a',
 'HRV_SD2d',
 'HRV_SD2a',
 'HRV_Cd',
 'HRV_Ca',
 'HRV_SDNNd',
 'HRV_SDNNa',
 'HRV_DFA_alpha1',
 'HRV_MFDFA_alpha1_Width',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_MFDFA_alpha1_Ma

In [177]:
# Apply the function to the dataframe
data30 = add_baseline_fatigue(data30)
data30

,HRV_PSS,HRV_Cd,HRV_ShanEn,HRV_Prc20NN,HRV_HFD,HRV_pNN50,HRV_SD2a,HRV_LnHF,Block,HRV_SD1SD2,...,HRV_C1a,HRV_Prc80NN,HRV_TINN,HRV_CVI,HRV_AI,HRV_MFDFA_alpha1_Asymmetry,HRV_ApEn,HRV_SDNNd,HRV_MaxNN,Baseline_Fatigue_Rating
11,0.961538,0.466627,4.506751,671.875,1.963142,70.270270,105.482183,-2.775916,1,1.003897,...,0.416144,903.125,109.375,5.438082,52.744979,-0.210564,0.117871,89.472374,1046.875,None
12,0.869565,0.498110,4.541311,656.250,1.996777,82.051282,143.982152,-2.560332,1,0.730381,...,0.320527,896.875,296.875,5.606783,54.055932,-0.892887,0.211589,115.054555,1093.750,None
13,0.960000,0.531885,3.901449,696.875,2.022870,64.102564,75.577070,-2.022340,1,0.844428,...,0.540641,796.875,312.500,5.267956,49.312868,-0.268321,0.323288,79.053999,953.125,None
14,0.960000,0.464495,3.895151,718.750,1.971984,54.054054,52.758316,-2.125872,1,1.217469,...,0.494456,840.625,78.125,4.958648,48.991938,-0.424693,0.125862,51.875308,984.375,None
15,0.962963,0.460136,4.041061,665.625,1.965879,53.846154,73.436613,-2.147839,1,1.067901,...,0.524776,828.125,218.750,5.218551,47.402338,-0.367628,0.348674,69.064922,1031.250,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0.958333,0.466440,2.712667,671.875,1.981808,14.285714,22.987323,-2.320926,6,0.813911,...,0.502415,718.750,15.625,4.094012,49.338920,-0.062091,0.357993,19.227614,765.625,None
491,0.928571,0.478795,3.213461,625.000,2.007681,17.391304,34.074239,-2.331859,6,0.812483,...,0.602369,671.875,125.000,4.508904,48.567483,-0.615376,0.143188,31.415539,718.750,None
492,1.000000,0.484253,3.326512,625.000,1.896967,27.272727,49.833781,-1.924997,6,0.923106,...,0.525105,687.500,156.250,4.858750,49.313554,-0.708718,0.258722,46.833030,953.125,None
493,0.888889,0.511616,3.998728,609.375,2.025425,55.813953,77.678462,-2.421838,6,1.023588,...,0.463160,750.000,62.500,5.283190,51.484606,-0.469674,0.502598,78.362242,1156.250,None


In [160]:
# f_path = "Only_HRVDelta_60seconds.xlsx"
f_path = "HRV_features_60seconds_withDelta.xlsx"
data60 = pd.read_excel(f_path, header=0)
data60.head()

,Unnamed: 0,Block,fatigue_rating,window_num_60,SubjID,Age,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,...,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC,Delta_fatigue,Change_from_Baseline
0,0,0,73,0,Cov4,38,709.821429,121.764550,NaN,NaN,...,1.447388,1.005172,1.122979,1.666676,2.003643,2.018747,3.009640,1.141485,0,0
1,1,0,73,1,Cov4,38,783.125000,92.079715,NaN,NaN,...,1.402135,1.027857,0.851633,0.971192,1.877574,2.003572,3.524105,1.245764,0,0
2,2,0,73,2,Cov4,38,767.248377,132.329919,NaN,NaN,...,1.324820,1.433895,1.078815,1.397637,2.553420,2.039852,3.455130,0.813868,0,0
3,3,0,73,3,Cov4,38,760.416667,111.418943,NaN,NaN,...,1.380736,1.167404,1.051312,1.380854,2.403422,2.019210,2.853212,1.128149,0,0
4,4,0,73,4,Cov4,38,762.419872,109.225866,NaN,NaN,...,1.198506,0.967043,0.996998,1.160986,1.736158,1.994097,2.905502,1.128149,0,0


In [161]:
def add_baseline_fatigue_and_delta(df):
    # Create a dictionary to hold the baseline fatigue rating for each subject
    baseline_fatigue_dict = {}

    # Loop through each subject and get their baseline fatigue rating (Block 0)
    for subj_id in df['SubjID'].unique():
        baseline_fatigue_values = df[(df['SubjID'] == subj_id) & (df['Block'] == 0)]['Fatigue_Rating'].values
        if len(baseline_fatigue_values) > 0:
            baseline_fatigue_dict[subj_id] = baseline_fatigue_values[0]
        else:
            baseline_fatigue_dict[subj_id] = None  # or set a default value if needed

    # Map the baseline fatigue rating to a new column in the dataframe
    df['Baseline_Fatigue_Rating'] = df['SubjID'].map(baseline_fatigue_dict)
    
    # Calculate Delta_fatigue and Change_from_Baseline for each row
    #df['Delta_fatigue'] = df.groupby('SubjID')['Fatigue_Rating'].diff().fillna(0)
    #df['Change_from_Baseline'] = df['Fatigue_Rating'] - df['Baseline_Fatigue_Rating']

    return df

In [162]:
# Apply the function to the dataframe
data60 = add_baseline_fatigue(data60)
data60.head()
data60.columns.tolist()

['Unnamed: 0',
 'Block',
 'fatigue_rating',
 'window_num_60',
 'SubjID',
 'Age',
 'HRV_MeanNN',
 'HRV_SDNN',
 'HRV_SDANN1',
 'HRV_SDNNI1',
 'HRV_SDANN2',
 'HRV_SDNNI2',
 'HRV_SDANN5',
 'HRV_SDNNI5',
 'HRV_RMSSD',
 'HRV_SDSD',
 'HRV_CVNN',
 'HRV_CVSD',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_MCVNN',
 'HRV_IQRNN',
 'HRV_SDRMSSD',
 'HRV_Prc20NN',
 'HRV_Prc80NN',
 'HRV_pNN50',
 'HRV_pNN20',
 'HRV_MinNN',
 'HRV_MaxNN',
 'HRV_HTI',
 'HRV_TINN',
 'HRV_ULF',
 'HRV_VLF',
 'HRV_LF',
 'HRV_HF',
 'HRV_VHF',
 'HRV_TP',
 'HRV_LFHF',
 'HRV_LFn',
 'HRV_HFn',
 'HRV_LnHF',
 'HRV_SD1',
 'HRV_SD2',
 'HRV_SD1SD2',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVI',
 'HRV_CSI_Modified',
 'HRV_PIP',
 'HRV_IALS',
 'HRV_PSS',
 'HRV_PAS',
 'HRV_GI',
 'HRV_SI',
 'HRV_AI',
 'HRV_PI',
 'HRV_C1d',
 'HRV_C1a',
 'HRV_SD1d',
 'HRV_SD1a',
 'HRV_C2d',
 'HRV_C2a',
 'HRV_SD2d',
 'HRV_SD2a',
 'HRV_Cd',
 'HRV_Ca',
 'HRV_SDNNd',
 'HRV_SDNNa',
 'HRV_DFA_alpha1',
 'HRV_MFDFA_alpha1_Width',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_

In [163]:
#for 30 second window data
# data30['SubjID'] = data30['SubjID'].str.replace('Cov', '')
# data30['SubjID'] = data30['SubjID'].astype(int)

data30 = data30.dropna(axis=1, how='all')
# data30 = data30[~data30.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
data30 = data30.loc[:, ~data30.isin([np.nan, np.inf, -np.inf]).any(axis=0)]

# data30['Window_ID'] = data30['SubjID'].map(str) + '_' + data30['Block'].map(str) + '_' + data30['window_num_30'].map(str)
# data30['Block_ID'] = data30['SubjID'].map(str) + '_' + data30['Block'].map(str)

block_zero = data30['Block'] == 0
data30 = data30[~block_zero]

#for 60 second window data
# data60['SubjID'] = data60['SubjID'].str.replace('Cov', '')
# data60['SubjID'] = data60['SubjID'].astype(int)

data60 = data60.dropna(axis=1, how='all')
data60 = data60.loc[:, ~data60.isin([np.nan, np.inf, -np.inf]).any(axis=0)]

# data60['Window_ID'] = data60['SubjID'].map(str) + '_' + data60['Block'].map(str) + '_' + data60['window_num_60'].map(str)
# data60['Block_ID'] = data60['SubjID'].map(str) + '_' + data60['Block'].map(str)

block_zero = data60['Block'] == 0
data60 = data60[~block_zero]

#for block level data
# data_block['SubjID'] = data_block['SubjID'].str.replace('Cov', '')
# data_block['SubjID'] = data_block['SubjID'].astype(int)

data_block = data_block.dropna(axis=1, how='all')
data_block = data_block.loc[:, ~data_block.isin([np.nan, np.inf, -np.inf]).any(axis=0)]

block_zero = data_block['Block'] == 0
data_block = data_block[~block_zero]

In [164]:
data_block

,Unnamed: 0,Block,SubjID,fatigue_rating,Age,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_RMSSD,...,Change_from_Baseline_HRV_SDRMSSD,Delta_HRV_Prc20NN,Change_from_Baseline_HRV_Prc20NN,Delta_HRV_HF,Change_from_Baseline_HRV_HF,Delta_HRV_HFD,Change_from_Baseline_HRV_HFD,Delta_HRV_LZC,Change_from_Baseline_HRV_LZC,Baseline_Fatigue_Rating
1,1,1,Cov4,82,38,767.857143,118.329781,34.008529,111.348149,155.844608,...,0.026449,15.625,15.625,0.001388,0.001388,-0.027743,-0.027743,-0.033584,-0.033584,73
2,2,2,Cov4,82,38,755.166997,117.852649,39.101073,111.234560,157.161219,...,0.017052,-15.625,0.000,-0.000836,0.000551,0.004485,-0.023258,-0.074824,-0.108408,73
3,3,3,Cov4,86,38,789.471419,136.276233,44.170195,125.032054,185.463881,...,0.001954,15.625,15.625,-0.002950,-0.002399,0.026965,0.003707,0.032553,-0.075854,73
4,4,4,Cov4,84,38,768.864829,114.462897,23.968917,112.300671,159.239493,...,-0.014022,0.000,15.625,-0.009351,-0.011750,-0.007814,-0.004107,0.028722,-0.047132,73
5,5,5,Cov4,86,38,797.528135,127.218396,51.129996,120.341598,174.286600,...,-0.002894,15.625,31.250,0.033100,0.021350,0.005814,0.001707,-0.027463,-0.074596,73
6,6,6,Cov4,84,38,774.772547,129.748274,33.032909,128.151493,155.922275,...,0.099302,-31.250,0.000,-0.028860,-0.007510,-0.047074,-0.045367,0.065939,-0.008656,73
8,8,1,Cov5,62,53,803.140470,169.386236,48.393441,162.970836,229.927034,...,-0.047100,46.875,46.875,0.080122,0.080122,0.014741,0.014741,-0.038965,-0.038965,55
9,9,2,Cov5,57,53,814.657928,168.692384,41.920998,157.682703,220.874527,...,-0.020048,15.625,62.500,-0.027563,0.052559,-0.003701,0.011040,0.027818,-0.011147,55
10,10,3,Cov5,59,53,811.387384,166.800627,41.373057,162.148192,211.684717,...,0.004172,0.000,62.500,-0.005614,0.046945,-0.019358,-0.008318,0.022968,0.011821,55
11,11,4,Cov5,69,53,831.008185,169.946696,37.256935,163.929312,225.471430,...,-0.030056,15.625,78.125,0.000864,0.047809,0.008064,-0.000254,-0.009013,0.002808,55


In [165]:
import pandas as pd

# List of column names of each DataFrame
cols_df1 = list(data30.columns)
cols_df2 = list(data60.columns)
cols_df3 = list(data_block.columns)
cols_df1 = [value.strip() for value in cols_df1]
cols_df2 = [value.strip() for value in cols_df2]
cols_df3 = [value.strip() for value in cols_df3]

# Find common columns across all DataFrames
common_columns = list(set(cols_df3).intersection(cols_df1, cols_df2))

# Retain only the common columns in each DataFrame
data30 = data30[common_columns]
data60 = data60[common_columns]
data_block = data_block[common_columns]
print(len(data30.columns), len(data60.columns), len(data_block.columns))
data30.columns.tolist()

72 72 72


['HRV_PSS',
 'HRV_Cd',
 'HRV_ShanEn',
 'HRV_Prc20NN',
 'HRV_HFD',
 'HRV_pNN50',
 'HRV_SD2a',
 'HRV_LnHF',
 'Block',
 'HRV_SD1SD2',
 'HRV_C2a',
 'Change_from_Baseline',
 'HRV_SD2',
 'HRV_SDSD',
 'HRV_HFn',
 'HRV_SDNN',
 'SubjID',
 'HRV_IALS',
 'HRV_LZC',
 'HRV_PAS',
 'HRV_MFDFA_alpha1_Increment',
 'HRV_PI',
 'HRV_SDRMSSD',
 'HRV_KFD',
 'HRV_MFDFA_alpha1_Fluctuation',
 'HRV_MFDFA_alpha1_Delta',
 'HRV_HF',
 'HRV_MFDFA_alpha1_Max',
 'HRV_pNN20',
 'HRV_CVSD',
 'HRV_C2d',
 'HRV_MeanNN',
 'HRV_HTI',
 'HRV_CD',
 'HRV_RMSSD',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_IQRNN',
 'HRV_TP',
 'HRV_MFDFA_alpha1_Width',
 'HRV_FuzzyEn',
 'HRV_CSI_Modified',
 'HRV_SD1d',
 'HRV_SI',
 'HRV_SD1a',
 'Age',
 'HRV_VHF',
 'HRV_C1d',
 'HRV_SDNNa',
 'HRV_GI',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_MCVNN',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVNN',
 'fatigue_rating',
 'HRV_MinNN',
 'HRV_DFA_alpha1',
 'HRV_PIP',
 'HRV_SD2d',
 'HRV_SD1',
 'HRV_Ca',
 'HRV_C1a',
 'HRV_Prc80NN',
 'HRV_TINN',
 'HRV_CVI',
 'HRV_AI'

In [166]:
data30

,HRV_PSS,HRV_Cd,HRV_ShanEn,HRV_Prc20NN,HRV_HFD,HRV_pNN50,HRV_SD2a,HRV_LnHF,Block,HRV_SD1SD2,...,HRV_Ca,HRV_C1a,HRV_Prc80NN,HRV_TINN,HRV_CVI,HRV_AI,HRV_MFDFA_alpha1_Asymmetry,HRV_ApEn,HRV_SDNNd,HRV_MaxNN
11,0.961538,0.466627,4.506751,671.875,1.963142,70.270270,105.482183,-2.775916,1,1.003897,...,0.533373,0.416144,903.125,109.375,5.438082,52.744979,-0.210564,0.117871,89.472374,1046.875
12,0.869565,0.498110,4.541311,656.250,1.996777,82.051282,143.982152,-2.560332,1,0.730381,...,0.501890,0.320527,896.875,296.875,5.606783,54.055932,-0.892887,0.211589,115.054555,1093.750
13,0.960000,0.531885,3.901449,696.875,2.022870,64.102564,75.577070,-2.022340,1,0.844428,...,0.468115,0.540641,796.875,312.500,5.267956,49.312868,-0.268321,0.323288,79.053999,953.125
14,0.960000,0.464495,3.895151,718.750,1.971984,54.054054,52.758316,-2.125872,1,1.217469,...,0.535505,0.494456,840.625,78.125,4.958648,48.991938,-0.424693,0.125862,51.875308,984.375
15,0.962963,0.460136,4.041061,665.625,1.965879,53.846154,73.436613,-2.147839,1,1.067901,...,0.539864,0.524776,828.125,218.750,5.218551,47.402338,-0.367628,0.348674,69.064922,1031.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0.958333,0.466440,2.712667,671.875,1.981808,14.285714,22.987323,-2.320926,6,0.813911,...,0.533560,0.502415,718.750,15.625,4.094012,49.338920,-0.062091,0.357993,19.227614,765.625
491,0.928571,0.478795,3.213461,625.000,2.007681,17.391304,34.074239,-2.331859,6,0.812483,...,0.521205,0.602369,671.875,125.000,4.508904,48.567483,-0.615376,0.143188,31.415539,718.750
492,1.000000,0.484253,3.326512,625.000,1.896967,27.272727,49.833781,-1.924997,6,0.923106,...,0.515747,0.525105,687.500,156.250,4.858750,49.313554,-0.708718,0.258722,46.833030,953.125
493,0.888889,0.511616,3.998728,609.375,2.025425,55.813953,77.678462,-2.421838,6,1.023588,...,0.488384,0.463160,750.000,62.500,5.283190,51.484606,-0.469674,0.502598,78.362242,1156.250


In [167]:
y_train30 = data30['Change_from_Baseline']
X_train30 = data30.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
X_train30 = X_train30.drop('SubjID', axis=1)

y_train60 = data60['Change_from_Baseline']
X_train60 = data60.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
X_train60 = X_train60.drop('SubjID', axis=1)

y_test = data_block['Change_from_Baseline']
X_test = data_block.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
X_test = X_test.drop('SubjID', axis=1)

print(len(X_train30.columns), len(X_train60.columns), len(X_test.columns))
X_train30.columns.tolist()

68 68 68


['HRV_PSS',
 'HRV_Cd',
 'HRV_ShanEn',
 'HRV_Prc20NN',
 'HRV_HFD',
 'HRV_pNN50',
 'HRV_SD2a',
 'HRV_LnHF',
 'HRV_SD1SD2',
 'HRV_C2a',
 'HRV_SD2',
 'HRV_SDSD',
 'HRV_HFn',
 'HRV_SDNN',
 'HRV_IALS',
 'HRV_LZC',
 'HRV_PAS',
 'HRV_MFDFA_alpha1_Increment',
 'HRV_PI',
 'HRV_SDRMSSD',
 'HRV_KFD',
 'HRV_MFDFA_alpha1_Fluctuation',
 'HRV_MFDFA_alpha1_Delta',
 'HRV_HF',
 'HRV_MFDFA_alpha1_Max',
 'HRV_pNN20',
 'HRV_CVSD',
 'HRV_C2d',
 'HRV_MeanNN',
 'HRV_HTI',
 'HRV_CD',
 'HRV_RMSSD',
 'HRV_MFDFA_alpha1_Peak',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_IQRNN',
 'HRV_TP',
 'HRV_MFDFA_alpha1_Width',
 'HRV_FuzzyEn',
 'HRV_CSI_Modified',
 'HRV_SD1d',
 'HRV_SI',
 'HRV_SD1a',
 'Age',
 'HRV_VHF',
 'HRV_C1d',
 'HRV_SDNNa',
 'HRV_GI',
 'HRV_MFDFA_alpha1_Mean',
 'HRV_MCVNN',
 'HRV_S',
 'HRV_CSI',
 'HRV_CVNN',
 'HRV_MinNN',
 'HRV_DFA_alpha1',
 'HRV_PIP',
 'HRV_SD2d',
 'HRV_SD1',
 'HRV_Ca',
 'HRV_C1a',
 'HRV_Prc80NN',
 'HRV_TINN',
 'HRV_CVI',
 'HRV_AI',
 'HRV_MFDFA_alpha1_Asymmetry',
 'HRV_ApEn',
 'HRV_SDNNd',
 'HRV

Leave one Out Validation While training with 30 second windows

In [168]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, r2_score

# Define features and target
sub_ids = data_block['SubjID'].unique()
print(sub_ids)
# Initialize Decision Tree classifier
bestModel = DecisionTreeRegressor()
min_rmse = 999999
for sub in sub_ids:
  print(sub)
  df_train = data30[data30['SubjID'] != sub]
  df_test = data_block[data_block['SubjID'] == sub]
  print(len(df_train), len(df_test))
  y_test = df_test['Change_from_Baseline']
  y_train = df_train['Change_from_Baseline']
  X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  X_train = X_train.drop('SubjID', axis=1)
  X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  X_test = X_test.drop('SubjID', axis=1)

  # Split the dataset into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Initialize Decision Tree classifier
  reg30 = DecisionTreeRegressor()

  # Train the classifier on the training data
  reg30.fit(X_train, y_train)

  # Make predictions on the testing data
  y_pred = reg30.predict(X_test)
  # Calculate the accuracy of the classifier
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Calculate the RMSE (Root Mean Squared Error)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE:", rmse)

  # Calculate the F1 score
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("F1 Score:", f1)

  r2 = r2_score(y_test, y_pred)
  print("R2 Score:", r2)
  if min_rmse > rmse:
    min_rmse = rmse
    bestModel = reg30

['Cov4' 'Cov5' 'Cov7' 'Cov8' 'Cov13' 'Cov14' 'Cov19' 'Cov22' 'Cov23']
Cov4
344 6
Accuracy: 0.0
RMSE: 37.69173207305107
F1 Score: 0.0
R2 Score: -531.75
Cov5
344 6
Accuracy: 0.16666666666666666
RMSE: 8.689073598491383
F1 Score: 0.06666666666666667
R2 Score: -2.060810810810811
Cov7
344 6
Accuracy: 0.0
RMSE: 36.08785575970583
F1 Score: 0.0
R2 Score: -4.942958549879579
Cov8
344 6
Accuracy: 0.16666666666666666
RMSE: 11.113055385446435
F1 Score: 0.1111111111111111
R2 Score: -0.9839357429718876
Cov13
389 3
Accuracy: 0.3333333333333333
RMSE: 5.94418483337567
F1 Score: 0.2222222222222222
R2 Score: -1.6065573770491803
Cov14
367 6
Accuracy: 0.16666666666666666
RMSE: 9.460443964212251
F1 Score: 0.05555555555555555
R2 Score: -0.1168110918544194
Cov19
346 6
Accuracy: 0.0
RMSE: 21.61403864775546
F1 Score: 0.0
R2 Score: -6.087231352718079
Cov22
396 6
Accuracy: 0.0
RMSE: 17.60208321004458
F1 Score: 0.0
R2 Score: -2.8945530726256985
Cov23
345 6
Accuracy: 0.0
RMSE: 11.430952132988164
F1 Score: 0.0
R2 Scor

In [ ]:
##random forest

In [176]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Select the feature columns (excluding target and non-feature columns)
feature_columns = data30.drop(['fatigue_rating', 'Change_from_Baseline', 'Block', 'SubjID'], axis=1).columns

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data30[feature_columns])
# Define features and target
sub_ids = data_block['SubjID'].unique()
print(sub_ids)
# Initialize Random Forest Regressor
bestModel = RandomForestRegressor(n_estimators=100, random_state=42)
min_rmse = 999999
for sub in sub_ids:
  print(sub)
  df_train = data30[data30['SubjID'] != sub]
  df_test = data_block[data_block['SubjID'] == sub]
  print(len(df_train), len(df_test))
  y_test = df_test['Change_from_Baseline']
  y_train = df_train['Change_from_Baseline']
  X_train =df_train[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_train =df_train[['Age', 'HRV_IQRNN', 'HRV_MedianNN', 'HRV_Prc80NN', 'HRV_SDNN', 'HRV_MCVNN', 'HRV_Prc20NN', 'HRV_pNN20', 'HRV_S', 'HRV_ShanEn']]
  #X_train =df_train[['HRV_HF', 'HRV_LnHF', 'HRV_IALS', 'HRV_PIP', 'HRV_Prc20NN', 'HRV_C1a', 'HRV_C1d', 'HRV_HFn', 'HRV_TP', 'HRV_MFDFA_alpha1_Delta']]
  # X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_train = X_train.drop('SubjID', axis=1)
  X_test =df_test[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_test =df_test[['Age', 'HRV_IQRNN', 'HRV_MedianNN', 'HRV_Prc80NN', 'HRV_SDNN', 'HRV_MCVNN', 'HRV_Prc20NN', 'HRV_pNN20', 'HRV_S', 'HRV_ShanEn']]
  #X_test =df_test[['HRV_HF', 'HRV_LnHF', 'HRV_IALS', 'HRV_PIP', 'HRV_Prc20NN', 'HRV_C1a', 'HRV_C1d', 'HRV_HFn', 'HRV_TP', 'HRV_MFDFA_alpha1_Delta']]
  # X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_test = X_test.drop('SubjID', axis=1)

  # Split the dataset into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Initialize Decision Tree classifier
  reg30 = DecisionTreeRegressor()

  # Train the classifier on the training data
  reg30.fit(X_train, y_train)

  # Make predictions on the testing data
  y_pred = reg30.predict(X_test)
  # Calculate the accuracy of the classifier
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Calculate the RMSE (Root Mean Squared Error)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE:", rmse)

  # Calculate the F1 score
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("F1 Score:", f1)

  r2 = r2_score(y_test, y_pred)
  print("R2 Score:", r2)
  if min_rmse > rmse:
    min_rmse = rmse
    bestModel = reg30

['Cov4' 'Cov5' 'Cov7' 'Cov8' 'Cov13' 'Cov14' 'Cov19' 'Cov22' 'Cov23']
Cov4
344 6
Accuracy: 0.0
RMSE: 36.97070912673076
F1 Score: 0.0
R2 Score: -511.5625
Cov5
344 6
Accuracy: 0.16666666666666666
RMSE: 8.698658900466592
F1 Score: 0.1111111111111111
R2 Score: -2.0675675675675675
Cov7
344 6
Accuracy: 0.0
RMSE: 37.63641853311763
F1 Score: 0.0
R2 Score: -5.46393712764609
Cov8
344 6
Accuracy: 0.16666666666666666
RMSE: 8.660254037844387
F1 Score: 0.05555555555555555
R2 Score: -0.20481927710843384
Cov13
389 3
Accuracy: 0.0
RMSE: 6.757711644237764
F1 Score: 0.0
R2 Score: -2.3688524590163937
Cov14
367 6
Accuracy: 0.16666666666666666
RMSE: 8.717797887081348
F1 Score: 0.05555555555555555
R2 Score: 0.051646447140381246
Cov19
346 6
Accuracy: 0.0
RMSE: 22.642143596988927
F1 Score: 0.0
R2 Score: -6.777496839443742
Cov22
396 6
Accuracy: 0.0
RMSE: 11.372481406154654
F1 Score: 0.0
R2 Score: -0.6256983240223464
Cov23
345 6
Accuracy: 0.0
RMSE: 7.571877794400365
F1 Score: 0.0
R2 Score: -0.10967741935483866


In [169]:
##train on reduced data

In [174]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, r2_score

# Define features and target
sub_ids = data_block['SubjID'].unique()
print(sub_ids)
# Initialize Decision Tree classifier
bestModelr = DecisionTreeRegressor()
min_rmse = 999999
for sub in sub_ids:
  print(sub)
  df_train = data30[data30['SubjID'] != sub]
  df_test = data_block[data_block['SubjID'] == sub]
  print(len(df_train), len(df_test))
  y_test = df_test['Change_from_Baseline']
  y_train = df_train['Change_from_Baseline']
  X_train =df_train[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_train =df_train[['Age', 'HRV_IQRNN', 'HRV_MedianNN', 'HRV_Prc80NN', 'HRV_SDNN', 'HRV_MCVNN', 'HRV_Prc20NN', 'HRV_pNN20', 'HRV_S', 'HRV_ShanEn']]
  #X_train =df_train[['HRV_HF', 'HRV_LnHF', 'HRV_IALS', 'HRV_PIP', 'HRV_Prc20NN', 'HRV_C1a', 'HRV_C1d', 'HRV_HFn', 'HRV_TP', 'HRV_MFDFA_alpha1_Delta']]
  # X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_train = X_train.drop('SubjID', axis=1)
  X_test =df_test[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_test =df_test[['Age', 'HRV_IQRNN', 'HRV_MedianNN', 'HRV_Prc80NN', 'HRV_SDNN', 'HRV_MCVNN', 'HRV_Prc20NN', 'HRV_pNN20', 'HRV_S', 'HRV_ShanEn']]
  #X_test =df_test[['HRV_HF', 'HRV_LnHF', 'HRV_IALS', 'HRV_PIP', 'HRV_Prc20NN', 'HRV_C1a', 'HRV_C1d', 'HRV_HFn', 'HRV_TP', 'HRV_MFDFA_alpha1_Delta']]
  # X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_test = X_test.drop('SubjID', axis=1)

  # Split the dataset into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Initialize Decision Tree classifier
  reg300 = DecisionTreeRegressor()

  # Train the classifier on the training data
  reg300.fit(X_train, y_train)

  # Make predictions on the testing data
  y_pred = reg300.predict(X_test)
  # Calculate the accuracy of the classifier
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Calculate the RMSE (Root Mean Squared Error)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE:", rmse)

  # Calculate the F1 score
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("F1 Score:", f1)

  r2 = r2_score(y_test, y_pred)
  print("R2 Score:", r2)
  if min_rmse > rmse:
    min_rmse = rmse
    bestModel = reg300

['Cov4' 'Cov5' 'Cov7' 'Cov8' 'Cov13' 'Cov14' 'Cov19' 'Cov22' 'Cov23']
Cov4
344 6
Accuracy: 0.0
RMSE: 37.5410886008029
F1 Score: 0.0
R2 Score: -527.5
Cov5
344 6
Accuracy: 0.16666666666666666
RMSE: 8.698658900466592
F1 Score: 0.1111111111111111
R2 Score: -2.0675675675675675
Cov7
344 6
Accuracy: 0.0
RMSE: 38.25354014118606
F1 Score: 0.0
R2 Score: -5.6776524274306
Cov8
344 6
Accuracy: 0.16666666666666666
RMSE: 8.660254037844387
F1 Score: 0.05555555555555555
R2 Score: -0.20481927710843384
Cov13
389 3
Accuracy: 0.3333333333333333
RMSE: 5.94418483337567
F1 Score: 0.16666666666666666
R2 Score: -1.6065573770491803
Cov14
367 6
Accuracy: 0.16666666666666666
RMSE: 9.495613022162743
F1 Score: 0.047619047619047616
R2 Score: -0.12512998266897757
Cov19
346 6
Accuracy: 0.0
RMSE: 22.642143596988927
F1 Score: 0.0
R2 Score: -6.777496839443742
Cov22
396 6
Accuracy: 0.0
RMSE: 11.372481406154654
F1 Score: 0.0
R2 Score: -0.6256983240223464
Cov23
345 6
Accuracy: 0.0
RMSE: 15.253414918196734
F1 Score: 0.0
R2 Sc

In [137]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, f1_score

# Assuming X_train, y_train are your training data and X_test_new, y_test_new are your testing data
# Train Decision Tree Regressor
tree_regressor = DecisionTreeRegressor()
# bestModel.fit(X_train30, y_train30)

# Predict on the testing data
y_pred = bestModel.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate the accuracy of the classifier
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

Accuracy: 0.16666666666666666
R2 Score: -1.9290322580645163
RMSE: 12.30176139149729
F1 Score: 0.16666666666666666


In [138]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Create and fit a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train30, y_train30)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X_train30.columns, 'Importance': feature_importances})

# Sort features by importance
feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)
# Print or visualize feature importances
for i in range(len(feature_importance)):
  print(feature_importance["Feature"][i], feature_importance_df["Importance"][i])

HRV_PSS 0.005212956929812732
HRV_Cd 0.0037471157293524036
HRV_ShanEn 0.00923761763592115
HRV_Prc20NN 0.018231221716562145
HRV_HFD 0.005756399980330867
HRV_pNN50 0.00449228711641351
HRV_SD2a 0.002753244020894766
HRV_LnHF 0.004999295234669046
HRV_SD1SD2 0.004802456822097534
HRV_C2a 0.0028392427382182596
HRV_SD2 0.007891381560899495
HRV_SDSD 0.001531442606521891
HRV_HFn 0.007210686356406003
HRV_SDNN 0.0076005470423081345
HRV_IALS 0.006175004813131646
HRV_LZC 0.007769809849914646
HRV_PAS 0.006319569531551583
HRV_MFDFA_alpha1_Increment 0.0034501998109828513
HRV_PI 0.009732498411911362
HRV_SDRMSSD 0.004619640103002969
HRV_KFD 0.00674530917424502
HRV_MFDFA_alpha1_Fluctuation 0.006658879274923068
HRV_MFDFA_alpha1_Delta 0.005006746057164013
HRV_HF 0.004116253258675653
HRV_MFDFA_alpha1_Max 0.008147439221018156
HRV_pNN20 0.009209790660437572
HRV_CVSD 0.004461107651193167
HRV_C2d 0.0034004871214951358
HRV_MeanNN 0.010266811634609845
HRV_HTI 0.004692447564974918
HRV_CD 0.009461656086760872
HRV_RMSS

In [139]:
# Select top 10 features
top_features = feature_importance['Feature'].head(10).tolist()
print("Top Features: ", top_features)

Top Features:  ['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']


In [149]:
from sklearn.feature_selection import mutual_info_regression

# Calculate mutual information scores
mi_scores = mutual_info_regression(X_train30, y_train30)

# Create a DataFrame to display feature names and their MI scores
mi_feature_importance_df = pd.DataFrame({'Feature': X_train30.columns, 'MI Score': mi_scores})

# Sort features by MI score
mi_feature_importance = mi_feature_importance_df.sort_values(by='MI Score', ascending=False)
print(mi_feature_importance)
# Select top 10 features
top_features = mi_feature_importance['Feature'].head(10).tolist()
print("Top Features: ", top_features)

                         Feature  MI Score
43                           Age  1.587178
35                     HRV_IQRNN  0.308769
33                  HRV_MedianNN  0.294375
60                   HRV_Prc80NN  0.289282
13                      HRV_SDNN  0.283880
..                           ...       ...
41                        HRV_SI  0.000000
16                       HRV_PAS  0.000000
17    HRV_MFDFA_alpha1_Increment  0.000000
65                      HRV_ApEn  0.000000
21  HRV_MFDFA_alpha1_Fluctuation  0.000000

[68 rows x 2 columns]
Top Features:  ['Age', 'HRV_IQRNN', 'HRV_MedianNN', 'HRV_Prc80NN', 'HRV_SDNN', 'HRV_MCVNN', 'HRV_Prc20NN', 'HRV_pNN20', 'HRV_S', 'HRV_ShanEn']


In [172]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Select the feature columns (excluding target and non-feature columns)
feature_columns = data30.drop(['fatigue_rating', 'Change_from_Baseline', 'Block', 'SubjID'], axis=1).columns

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data30[feature_columns])

# Apply PCA
n_components = 10  # Number of components you want to keep
pca = PCA(n_components=n_components)
pca.fit(X_scaled)

# Transform the data
X_pca = pca.transform(X_scaled)

# Get the loadings (contributions of each feature to the components)
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(n_components)], index=feature_columns)

# Calculate the cumulative importance of each feature across all principal components
cumulative_importance = loadings.abs().sum(axis=1)

# Select the top 10 features based on cumulative importance
top_10_features = cumulative_importance.sort_values(ascending=False).head(10).index.tolist()

# Display the top 10 features
print("Top 10 features based on PCA:", top_10_features)

Top 10 features based on PCA: ['HRV_HF', 'HRV_LnHF', 'HRV_IALS', 'HRV_PIP', 'HRV_Prc20NN', 'HRV_C1a', 'HRV_C1d', 'HRV_HFn', 'HRV_TP', 'HRV_MFDFA_alpha1_Delta']


Leave one out Validation while training on 60 second windows

In [145]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, r2_score

# Define features and target
sub_ids = data_block['SubjID'].unique()
print(sub_ids)
# Initialize Decision Tree classifier
bestModel = DecisionTreeRegressor()
min_rmse = 999999
for sub in sub_ids:
  print(sub)
  df_train = data60[data60['SubjID'] != sub]
  df_test = data_block[data_block['SubjID'] == sub]
  print(len(df_train), len(df_test))
  y_test = df_test['Change_from_Baseline']
  y_train = df_train['Change_from_Baseline']
  X_train =df_train[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_train = X_train.drop('SubjID', axis=1)
  X_test =df_test[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
  # X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline', 'Block'], axis=1)
  # X_test = X_test.drop('SubjID', axis=1)

  # Split the dataset into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Initialize Decision Tree classifier
  reg60 = DecisionTreeRegressor()

  # Train the classifier on the training data
  reg60.fit(X_train, y_train)

  # Make predictions on the testing data
  y_pred = reg60.predict(X_test)
  # Calculate the accuracy of the classifier
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Calculate the RMSE (Root Mean Squared Error)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE:", rmse)

  # Calculate the F1 score
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("F1 Score:", f1)

  r2 = r2_score(y_test, y_pred)
  print("R2 Score:", r2)
  if min_rmse > rmse:
    min_rmse = rmse
    bestModel = reg60

['Cov4' 'Cov5' 'Cov7' 'Cov8' 'Cov13' 'Cov14' 'Cov19' 'Cov22' 'Cov23']
Cov4
166 6
Accuracy: 0.0
RMSE: 16.170961628796228
F1 Score: 0.0
R2 Score: -97.0625
Cov5
164 6
Accuracy: 0.0
RMSE: 15.253414918196734
F1 Score: 0.0
R2 Score: -8.432432432432432
Cov7
166 6
Accuracy: 0.0
RMSE: 33.01009946465879
F1 Score: 0.0
R2 Score: -3.9724933451641533
Cov8
166 6
Accuracy: 0.0
RMSE: 12.267844146385297
F1 Score: 0.0
R2 Score: -1.4176706827309236
Cov13
184 3
Accuracy: 0.0
RMSE: 7.615773105863909
F1 Score: 0.0
R2 Score: -3.278688524590164
Cov14
178 6
Accuracy: 0.0
RMSE: 9.38971068066885
F1 Score: 0.0
R2 Score: -0.10017331022530329
Cov19
167 6
Accuracy: 0.0
RMSE: 24.876360934295302
F1 Score: 0.0
R2 Score: -8.388116308470291
Cov22
187 6
Accuracy: 0.0
RMSE: 9.755340417774597
F1 Score: 0.0
R2 Score: -0.19622905027932958
Cov23
167 6
Accuracy: 0.0
RMSE: 12.13122692338523
F1 Score: 0.0
R2 Score: -1.8483870967741933


In [143]:
# import pandas as pd
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import LeaveOneGroupOut
# from sklearn.metrics import accuracy_score, r2_score


# # Define features and target
# sub_ids = data_block['SubjID'].unique()
# print(sub_ids)
# # Initialize Decision Tree classifier
# bestModel = DecisionTreeRegressor()

# for sub in sub_ids:
#   print(sub)
#   df_train = data60[data60['SubjID'] != sub]
#   df_test = data_block[data_block['SubjID'] == sub]
#   # print(len(df))
#   y_test = df_test['Change_from_Baseline']
#   y_train = df_train['Change_from_Baseline']
#   X_train =df_train[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
#   # X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline','Delta_fatigue', 'Block'], axis=1)
#   # X_train = X_train.drop('SubjID', axis=1)
#   X_test =df_test[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
#   # X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline','Delta_fatigue', 'Block'], axis=1)
#   # X_test = X_test.drop('SubjID', axis=1)

#   # Split the dataset into training and testing sets
#   # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#   # Train the classifier on the training data
#   reg60.fit(X_train, y_train)

#   # Make predictions on the testing dataimport pandas as pd


# # Define features and target
# sub_ids = data_block['SubjID'].unique()
# print(sub_ids)
# # Initialize Decision Tree classifier
# bestModel = DecisionTreeRegressor()
# min_rmse = 999999
# for sub in sub_ids:
#   print(sub)
#   df_train = data60[data60['SubjID'] != sub]
#   df_test = data_block[data_block['SubjID'] == sub]
#   print(len(df_train), len(df_test))
#   y_test = df_test['Change_from_Baseline']
#   y_train = df_train['Change_from_Baseline']
#   X_train =df_train[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
#   # X_train = df_train.drop(['fatigue_rating', 'Change_from_Baseline','Delta_fatigue', 'Block'], axis=1)
#   # X_train = X_train.drop('SubjID', axis=1)
#   X_test =df_test[['Age', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_MaxNN', 'HRV_Prc20NN', 'HRV_SDNNd', 'HRV_SD2d', 'HRV_MeanNN', 'HRV_PI']]
#   # X_test = df_test.drop(['fatigue_rating', 'Change_from_Baseline','Delta_fatigue', 'Block'], axis=1)
#   # X_test = X_test.drop('SubjID', axis=1)

#   # Split the dataset into training and testing sets
#   # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#   # Initialize Decision Tree classifier
#   reg60 = DecisionTreeRegressor()

#   # Train the classifier on the training data
#   reg60.fit(X_train, y_train)

#   # Make predictions on the testing data
#   y_pred = reg60.predict(X_test)
#   # Calculate the accuracy of the classifier
#   accuracy = accuracy_score(y_test, y_pred)
#   print("Accuracy:", accuracy)

#   # Calculate the RMSE (Root Mean Squared Error)
#   rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#   print("RMSE:", rmse)

#   # Calculate the F1 score
#   f1 = f1_score(y_test, y_pred, average='weighted')
#   print("F1 Score:", f1)

#   r2 = r2_score(y_test, y_pred)
#   print("R2 Score:", r2)
#   if min_rmse > rmse:
#     min_rmse = rmse
#     bestModel = reg60

['Cov4' 'Cov5' 'Cov7' 'Cov8' 'Cov13' 'Cov14' 'Cov19' 'Cov22' 'Cov23']
Cov4


NameError: name 'reg60' is not defined

In [90]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, f1_score

# X_train, y_train are your training data and X_test_new, y_test_new are your testing data
# Train Decision Tree Regressor
# tree_regressor = DecisionTreeRegressor()
# bestModel.fit(X_train60, y_train60)

# Predict on the testing data
y_pred = bestModel.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

Accuracy: 0.0
R2 Score: -3.2709677419354835
RMSE: 14.854853303438128
F1 Score: 0.0


In [91]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Create and fit a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train60, y_train60)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X_train60.columns, 'Importance': feature_importances})

# Sort features by importance
feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)
# Print or visualize feature importances
for i in range(len(feature_importance)):
  print(feature_importance["Feature"][i], feature_importance_df["Importance"][i])

HRV_CVNN 0.008823349183310687
HRV_KFD 0.010178299868604499
HRV_DFA_alpha1 0.007376281827528847
HRV_MeanNN 0.009218253218251088
HRV_CD 0.004545774705373232
HRV_GI 0.0027811707041992964
HRV_Cd 0.002857338376578908
HRV_MaxNN 0.0032671616691264185
HRV_C1d 0.004942918345063104
HRV_MFDFA_alpha1_Peak 0.009095235047665157
HRV_CVI 0.008558675744470537
HRV_MFDFA_alpha1_Asymmetry 0.01452379263703764
HRV_ApEn 0.00980508041886255
HRV_RMSSD 0.0034368539139743117
HRV_IALS 0.005007176815509627
HRV_SD1 0.0016306562315277218
HRV_PAS 0.010668245322523435
HRV_PIP 0.009260168788078569
HRV_SD2a 0.001633918222467014
HRV_TP 0.005370809500250197
HRV_SDNNa 0.0042204443369714085
HRV_C2a 0.003907196991424897
HRV_SI 0.006827977971828379
Age 0.3579304154272171
HRV_HTI 0.006274514545321088
HRV_MadNN 0.017165589764690403
HRV_Prc80NN 0.01095548623577175
HRV_PI 0.005870762137154909
HRV_MFDFA_alpha1_Fluctuation 0.005617749891820798
HRV_MFDFA_alpha1_Mean 0.007927729802914065
HRV_CSI_Modified 0.0071074607185318325
HRV_PSS

In [92]:
feature_importance

,Feature,Importance
23,Age,0.357930
37,HRV_ShanEn,0.081030
65,HRV_MCVNN,0.064639
59,HRV_SD2d,0.037218
63,HRV_pNN20,0.030840
...,...,...
36,HRV_HF,0.002156
58,HRV_CSI,0.001865
44,HRV_MinNN,0.001652
18,HRV_SD2a,0.001634
